<a href="https://colab.research.google.com/github/TEAM-N0VA/strat-final/blob/main/colab_server/logic/analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =========================================================
# [1] 필수 라이브러리 설치 및 임포트
# =========================================================
!pip install flask-ngrok flask pyngrok pandas scikit-learn

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from flask import Flask, request, jsonify
from pyngrok import ngrok
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

# =========================================================
# [2] 데이터 전처리
# =========================================================
def format_usage(value, category):
    """카테고리에 따라 시간(분) 또는 횟수(회)로 포맷팅"""
    if category in ['SHOPPING', 'FOOD']:
        return f"{int(value)}회"
    else:
        minutes = int(value)
        if minutes < 60: return f"{minutes}분"
        hours = minutes // 60
        mins = minutes % 60
        if mins == 0: return f"{hours}시간"
        return f"{hours}시간 {mins}분"

def generate_population_data(n=1000):
    """AI 학습용 비교군 데이터 생성 """
    np.random.seed(42)
    n_heavy = int(n * 0.15); n_normal = int(n * 0.55); n_waste = int(n * 0.30)

    # 1. 시간 기반
    pop_time_heavy = pd.DataFrame({'usage_amount': np.random.normal(3000, 600, n_heavy), 'category': 'OTT'})
    pop_time_normal = pd.DataFrame({'usage_amount': np.random.normal(1200, 300, n_normal), 'category': 'OTT'})
    pop_time_waste = pd.DataFrame({'usage_amount': np.random.randint(0, 300, n_waste), 'category': 'OTT'})

    # 2. 횟수 기반
    pop_count_heavy = pd.DataFrame({'usage_amount': np.random.randint(10, 16, n_heavy), 'category': 'SHOPPING'})
    pop_count_normal = pd.DataFrame({'usage_amount': np.random.randint(6, 10, n_normal), 'category': 'SHOPPING'})
    pop_count_waste = pd.DataFrame({'usage_amount': np.random.randint(0, 4, n_waste), 'category': 'SHOPPING'})

    pop = pd.concat([pop_time_heavy, pop_time_normal, pop_time_waste, pop_count_heavy, pop_count_normal, pop_count_waste], ignore_index=True)
    pop['usage_amount'] = pop['usage_amount'].apply(lambda x: max(0, x))
    pop['cost'] = 10000
    return pop

# =========================================================
# [2.5] MockDataGenerator
# =========================================================
class MockDataGeneratorPython:

    def __init__(self):
        self.TIME_BASED = "TIME_BASED"
        self.COUNT_BASED = "COUNT_BASED"

        # 구독 서비스 정의
        self.subscriptions = [
            {"name": "넷플릭스", "pkg": "com.netflix", "cost": 13500, "type": self.TIME_BASED, "cat": "OTT"},
            {"name": "유튜브 프리미엄", "pkg": "com.google.youtube", "cost": 14900, "type": self.TIME_BASED, "cat": "OTT"},
            {"name": "멜론", "pkg": "com.iloen.melon", "cost": 10900, "type": self.TIME_BASED, "cat": "MUSIC"},
            {"name": "배민클럽", "pkg": "com.woowahan.baemin", "cost": 3990, "type": self.COUNT_BASED, "cat": "FOOD"},
            {"name": "쿠팡와우", "pkg": "com.coupang.mobile", "cost": 4990, "type": self.COUNT_BASED, "cat": "SHOPPING"}
        ]

    def generate(self):
        mock_list = []
        today = datetime.now()

        # 1. 효율/비효율 랜덤 배정 (3개는 효율, 나머지는 비효율)
        indices = list(range(len(self.subscriptions)))
        efficient_indices = set(random.sample(indices, 3))

        for index, sub in enumerate(self.subscriptions):
            is_efficient = index in efficient_indices

            # (A) 결제 로그 (25~30일 전 1회)
            days_ago_pay = random.randint(25, 30)
            pay_date = (today - timedelta(days=days_ago_pay)).strftime("%Y%m%d")

            mock_list.append({
                "date": pay_date, "serviceName": sub['name'], "cost": sub['cost'],
                "timeMinutes": 0, "payment_count": 1, "logType": "SUB_PAYMENT", "category": sub['cat']
            })

            # (B) 사용 로그 (최근 30일) - 확률 로직 적용
            for day in range(1, 31):
                usage_date = (today - timedelta(days=day)).strftime("%Y%m%d")

                # 확률 로직
                if sub['type'] == self.TIME_BASED:
                    prob = 0.8 if is_efficient else 0.1
                else:
                    prob = 0.28 if is_efficient else 0.05

                if random.random() < prob:
                    minutes = 0; count = 0
                    if sub['type'] == self.TIME_BASED:
                        minutes = random.randint(60, 180) if is_efficient else random.randint(5, 15)
                        count = 1
                    else:
                        minutes = 10; count = 1 # 횟수 기반

                    mock_list.append({
                        "date": usage_date, "serviceName": sub['name'], "cost": 0,
                        "timeMinutes": minutes, "payment_count": count, "logType": "SUB_USAGE", "category": sub['cat']
                    })

        return pd.DataFrame(mock_list)

# =========================================================
# [3] 핵심 AI 분석 로직
# =========================================================
def analyze_advanced_ai(user_logs_df):
    if user_logs_df.empty: return create_empty_response()

    cols = ['cost', 'timeMinutes', 'payment_count']
    for c in cols:
        user_logs_df[c] = pd.to_numeric(user_logs_df[c], errors='coerce').fillna(0)

    # 1. 파이차트용
    pie_df = user_logs_df.copy()

    # 2. 효율 분석용
    target_subscriptions = ["넷플릭스", "유튜브 프리미엄", "멜론", "배민클럽", "쿠팡와우", "티빙", "디즈니플러스", "요기요"]
    mask = user_logs_df['serviceName'].isin(target_subscriptions)
    sub_raw_df = user_logs_df[mask].copy()

    if sub_raw_df.empty: return create_response(pie_df, [], None, {}, "분석할 구독 서비스가 없습니다.")

    # 3. 데이터 집계
    my_agg_df = sub_raw_df.groupby(['serviceName', 'category']).agg({
        'cost': 'sum', 'timeMinutes': 'sum', 'payment_count': 'sum'
    }).reset_index()

    my_agg_df['usage_amount'] = my_agg_df.apply(
        lambda x: x['payment_count'] if x['category'] in ['SHOPPING', 'FOOD'] else x['timeMinutes'], axis=1
    )

    # --- AI 모델링 ---
    pop_df = generate_population_data()
    features = ['usage_amount']
    pop_df['is_user'] = False; my_agg_df['is_user'] = True
    combined = pd.concat([pop_df[features + ['is_user']], my_agg_df[features + ['is_user']]], ignore_index=True)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(combined[features])

    km = KMeans(n_clusters=3, random_state=42, n_init=10)
    combined['cluster'] = km.fit_predict(X_scaled)
    c_means = combined.groupby('cluster')['usage_amount'].mean().sort_values()
    sorted_clusters = c_means.index.tolist()

    iso = IsolationForest(contamination=0.05, random_state=42)
    combined['anomaly'] = iso.fit_predict(X_scaled)

    my_results = combined[combined['is_user'] == True].copy().reset_index(drop=True)
    my_agg_df.reset_index(drop=True, inplace=True)
    final_df = pd.concat([my_agg_df, my_results[['cluster', 'anomaly']]], axis=1)

    pop_only = combined[combined['is_user'] == False]
    avg_time = pop_only[pop_only['usage_amount'] > 100]['usage_amount'].mean()
    avg_count = pop_only[pop_only['usage_amount'] <= 100]['usage_amount'].mean()

    monthly_report = []
    realtime_status = {}

    for i, row in final_df.iterrows():
        usage = row['usage_amount']; cat = row['category']
        safe_avg = avg_count if cat in ['SHOPPING', 'FOOD'] else avg_time
        if safe_avg == 0: safe_avg = 1
        pct_diff = int(((usage - safe_avg) / safe_avg) * 100)
        comp_text = f"평균 대비 {abs(pct_diff)}% " + ("상회" if pct_diff >= 0 else "하회")

        rank = sorted_clusters.index(row['cluster'])
        if pct_diff > 100: rank = 2
        elif pct_diff >= 0 and rank == 0: rank = 1

        status = "합리적 소비"; alert = "GOOD"
        limit_red_time = 60; limit_yellow_time = 300
        limit_red_count = 2; limit_yellow_count = 5
        is_red = False; is_yellow = False

        if cat in ['SHOPPING', 'FOOD']:
            if usage < limit_red_count: is_red = True
            elif usage < limit_yellow_count: is_yellow = True
        else:
            if usage < limit_red_time: is_red = True
            elif usage < limit_yellow_time: is_yellow = True

        if is_red: status = "🔴 매우 비효율"; alert = "WARNING"; comp_text = "사용 이력 거의 없음"
        elif is_yellow: status = "🟡 관심 필요"; alert = "CAUTION"; comp_text = "활용도 다소 낮음"
        elif rank == 1 or (not is_red and not is_yellow and rank == 0): status = "🟢 합리적 소비"; alert = "GOOD"
        elif rank == 2:
            status = "🔵 최상의 효율"; alert = "BEST"
            if row['anomaly'] == -1: status = "💎 상위 1% 효율"

        monthly_report.append({
            "service": row['serviceName'], "status": status, "message": comp_text,
            "detail": f"월 {format_usage(usage, cat)} 사용", "alert_level": alert
        })

        target = 1000 if cat not in ['SHOPPING', 'FOOD'] else 10
        realtime_status[row['serviceName']] = {
            "remaining": int(max(target - usage, 0)), "unit": "회" if cat in ['SHOPPING', 'FOOD'] else "분",
            "percent": int(min((usage / target) * 100, 100))
        }

    return create_response(pie_df, monthly_report, my_agg_df, realtime_status)

# =========================================================
# [4] 응답 생성
# =========================================================
def create_response(raw_df, monthly_report, agg_df, realtime_status, rec_msg=None):
    pie_chart = []
    if not raw_df.empty:
        grp = raw_df.groupby('category')['cost'].sum().reset_index()
        total = grp['cost'].sum()
        if total > 0:
            grp_sorted = grp.sort_values(by='cost', ascending=False)
            for i, r in grp_sorted.iterrows():
                pie_chart.append({"category": r['category'], "amount": int(r['cost']), "percent": round(r['cost']/total*100, 1)})

    persona = "⚖️ 밸런스형 스마트 컨슈머"
    if not raw_df.empty and 'cost' in raw_df.columns and raw_df['cost'].sum() > 0:
        max_cat = raw_df.groupby('category')['cost'].sum().idxmax()
        if max_cat == 'OTT': persona = "🍿 콘텐츠 탐험가"
        elif max_cat == 'FOOD': persona = "🍽️ 맛잘알 미식가"
        elif max_cat == 'SHOPPING': persona = "🛍️ 쇼핑 마스터"
        elif max_cat == 'MUSIC': persona = "🎧 감성 리스너"

    if rec_msg is None:
        warning_subs = [x for x in monthly_report if x['alert_level'] == 'WARNING']
        caution_subs = [x for x in monthly_report if x['alert_level'] == 'CAUTION']
        if warning_subs:
            names = ", ".join([w['service'] for w in warning_subs[:2]])
            rec_msg = f"현재 '{names}' 사용 효율이 매우 낮습니다. 해지를 고려해보세요."
        elif caution_subs:
            rec_msg = f"'{caution_subs[0]['service']}'의 활용도가 아쉽네요. 더 자주 이용해보세요!"
        else:
            rec_msg = f"{persona}님, 빈틈없는 소비 관리 능력이 돋보입니다!"

    weekly_report = []
    if agg_df is not None and not agg_df.empty:
        for i, row in agg_df.iterrows():
            weekly_avg = int(row['usage_amount'] / 4)
            status_msg = "합리적 사용"; alert = "GOOD"
            for m in monthly_report:
                if m['service'] == row['serviceName']:
                    alert = m['alert_level']; break
            weekly_report.append({
                "service": row['serviceName'], "usage_this_week": weekly_avg,
                "message": f"{format_usage(weekly_avg, row['category'])}/주", "alert_level": alert
            })

    return {
        "persona": persona, "monthly_report": monthly_report, "weekly_report": weekly_report,
        "realtime_status": realtime_status, "pie_chart": pie_chart, "recommendation": rec_msg
    }

def create_empty_response():
    return {"persona": "데이터 없음", "monthly_report": [], "recommendation": "데이터가 없습니다."}

# =========================================================
# [5] 실행
# =========================================================
app = Flask(__name__)

@app.route('/analyze', methods=['POST'])
def analyze():
    try:
        data = request.json
        logs = data.get('logs', [])
        df = pd.DataFrame(logs)
        result = analyze_advanced_ai(df)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':

    # ------------------------------------------------------------------
    # 영상 촬영용
    # ------------------------------------------------------------------
    print("="*70)
    print("🎬 [Integrated Demo] 안드로이드 앱 데이터 생성 -> AI 분석 -> 결과 리포트")
    print("="*70)

    # 1. Input: 안드로이드 로직으로 데이터 생성
    print("\n📱 [Step 1] Android Mock Data Generation")
    print(">> 안드로이드 앱 내부의 'MockDataGenerator' 로직을 실행합니다...")
    gen = MockDataGeneratorPython()
    mock_df = gen.generate()

    # 생성된 데이터 일부
    print(f">> 생성된 로그 개수: {len(mock_df)}건")
    print(mock_df[['date', 'serviceName', 'category', 'timeMinutes', 'payment_count']].head(7).to_string(index=False))
    print("... (생략) ...")

    # 2. Process: AI 엔진 분석
    print("\n⚙️ [Step 2] Sending Data to AI Engine & Processing")
    print(">> 생성된 로그를 AI 분석 함수(analyze_advanced_ai)에 주입합니다.")
    ai_result = analyze_advanced_ai(mock_df)

    # 3. Output: 안드로이드 화면에 보여질 결과 출력
    print("\n✅ [Step 3] AI Analysis Result (User View)")

    print("\n[1] 📊 Monthly Report (월간 분석)")
    print("-" * 70)
    print(f"👤 User Persona: {ai_result['persona']}")
    print("-" * 70)
    print(f"{'Service':<12} | {'Status':<12} | {'Alert':<8} | {'Details'}")
    print("-" * 70)
    for item in ai_result['monthly_report']:
        print(f"{item['service']:<12} | {item['status']:<12} | {item['alert_level']:<8} | {item['detail']}")

    # 주간 리포트 출력
    print("\n[2] 📅 Weekly Report (주간 분석)")
    print("-" * 70)
    if ai_result['weekly_report']:
        print(f"{'Service':<12} | {'Weekly Avg':<15} | {'Message'}")
        print("-" * 70)
        for item in ai_result['weekly_report']:
            # 단순 숫자와 포맷팅된 메시지 함께 출력
            print(f"{item['service']:<12} | {str(item['usage_this_week']):<15} | {item['message']}")
    else:
        print("주간 데이터 없음")

    print("-" * 70)
    print(f"💡 Recommendation: {ai_result['recommendation']}")
    print("="*70)


🎬 [Integrated Demo] 안드로이드 앱 데이터 생성 -> AI 분석 -> 결과 리포트

📱 [Step 1] Android Mock Data Generation
>> 안드로이드 앱 내부의 'MockDataGenerator' 로직을 실행합니다...
>> 생성된 로그 개수: 58건
    date serviceName category  timeMinutes  payment_count
20251120        넷플릭스      OTT            0              1
20251218        넷플릭스      OTT          121              1
20251217        넷플릭스      OTT          114              1
20251216        넷플릭스      OTT          123              1
20251215        넷플릭스      OTT           72              1
20251214        넷플릭스      OTT          157              1
20251213        넷플릭스      OTT          117              1
... (생략) ...

⚙️ [Step 2] Sending Data to AI Engine & Processing
>> 생성된 로그를 AI 분석 함수(analyze_advanced_ai)에 주입합니다.

✅ [Step 3] AI Analysis Result (User View)

[1] 📊 Monthly Report (월간 분석)
----------------------------------------------------------------------
👤 User Persona: 🍿 콘텐츠 탐험가
----------------------------------------------------------------------
Service      | Statu